# Ngram language model lab

In this lab you will do 4 exercises building and evaluating ngram language models of the following types:
1. A Maximum Liklihood Estimation (MLE) unigram model (10 marks)
2. A bigram model with add-one Laplace smoothing (10 marks)
3. A bigram model with general additive (add-k) smoothing (10 marks)
4. Ngram models with an advanced interpolation technique, Kneser-Ney snoothing (the methods are provided) (10 marks)

Before you start the exercises, make sure you run and understand the examples first. Then complete the exercises using the following 3 files with line-separated text to train the bigger language models on:
* training data -- "switchboard_lm_training.txt"
* heldout/development data -- "switchboard_lm_heldout.txt"
* test data -- "switchboard_lm_test.txt"

In [1]:
from __future__ import division  # for python 2 this is needed
from __future__ import print_function # for python 2 this is needed
from collections import Counter
from math import log

In [2]:
# Some useful methods for all exercises:
def glue_tokens(tokens, order):
    """A useful way of glueing tokens together for
    Kneser Ney smoothing and other smoothing methods
    
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    return '{0}@{1}'.format(order,' '.join(tokens))

def unglue_tokens(tokenstring, order):
    """Ungluing tokens glued by the glue_tokens method"""
    if order == 1:
        return [tokenstring.split("@")[1].replace(" ","")]
    return tokenstring.split("@")[1].split(" ")

def tokenize_sentence(sentence, order):
    """Returns a list of tokens with the correct numbers of initial
    and end tags (this is meant ot be used with a non-backoff model!!!)
    
    :sentence: a string of text
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    tokens = sentence.split()
    tokens = ['<s>'] * (order-1) + tokens + ['</s>']
    return tokens

# Examples

In [3]:
# Set of sentences (corpus) from the example in the lecture slides
sentences = [
            "I am Sam",
            "Sam I am",
            "I do not like green eggs and ham"
            ]

## Example 1. Build a unigram MLE language model from a simple corpus.
An MLE unigram model will tell you how likely a word is to occur, estimated from the function of counts:

C(w_i)/N

where C(w_i) is the number of times the word at position i occurred in the training corpus, and N is the sum of the counts of all words, or, to put it another way, the length of the training corpus.

Notice the tokenization method adds a `</s>` at the end but no `<s>` is needed at the beginning of each sentence
    because unigrams do not have a context word (we are only concerned with the frequency of single words).

In [4]:
unigrams = Counter()
for sent in sentences:
    words = tokenize_sentence(sent, 1)
    print("tokenized", words)
    for w in words:
        unigrams[w] +=1
unigram_total = sum(unigrams.values()) # to get the denominator for unigram probabilities
print (len(unigrams), "different unigrams observed")
print("unigram total", unigram_total)

tokenized ['I', 'am', 'Sam', '</s>']
tokenized ['Sam', 'I', 'am', '</s>']
tokenized ['I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>']
11 different unigrams observed
unigram total 17


To evaluate the model, we will measure the model's perplexity of those same training sentences. Note in normal practice you would want to do this on different data (as you will do below).

Perplexity is equal to 2 to the power of the cross entropy where cross entropy is the negative sum of all log probabilities from the model normalized by the length of the corpus N.

Measure the cross entropy and perplexity on each sentence too.

In [5]:
s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s 
for sent in sentences:
    # get the unigram model based probability of each sentence
    words = tokenize_sentence(sent, 1) # tokenize sentence with the order 1 as the parameter
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for w in words:
        prob = unigrams[w]/unigram_total
        logprob = log(prob, 2)  # the log of the prob to base 2
        s += -log(prob, 2) # add the neg log prob to s
        sent_s += -log(prob, 2)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N
    sent_perplexity = 2 ** sent_cross_entropy
    print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print("unigram corpus cross entropy", cross_entropy)
print("unigram corpus perplexity", perplexity)

['I', 'am', 'Sam', '</s>'] cross entropy: 2.79498159089 perplexity: 6.94022093789
['Sam', 'I', 'am', '</s>'] cross entropy: 2.79498159089 perplexity: 6.94022093789
['I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>'] cross entropy: 3.7352489522 perplexity: 13.3174776279
unigram corpus cross entropy 3.29277019394
unigram corpus perplexity 9.79992150705


## Example 2. Build a bigram MLE language model from the same corpus
A MLE (unsmoothed) bigram model will tell you how likely a word is to occur given the previous word, estimated from the function of counts:

C(w_i-1, w_i)/C(w_i-1)

where for any pairs of contiguous words w_i-1, w_i, C(w_i-1, w_i) is the number of times the word at position i follows the word at position i-1 in the training corpus, and C(w_i-1) is the number of times the word at position i-1 occurs in the corpus. E.g. for the bigram probability of 'john likes', C(w_i-1, w_i) is the number of times 'john likes' occurs, and C(w_i-1) is how many times 'john' occurs.

Notice the tokenization method adds a `</s>` at the end and also one `<s>` for padding at the beginning as we want to count the number of times the word at the beginning of each sentence begins a sentence.

In [6]:
# First get the counts from the training corpus for bigrams without smoothing
bigrams = Counter() # a counter for how many times a given bigram sequence w_i-1,w_i occurs
bigram_context = Counter() # a counter for how many times each word is used as a context word w_i-1 (so will include the start symbol)
delta = 1  # delta is order - 1
for s in sentences:
    words = tokenize_sentence(s, 2)  # tokenize sentence with the order 2 as the parameter
    for i in range(delta, len(words)):
        context = words[i-delta:i]
        target = words[i]
        ngram = context + [target]
        bigrams[glue_tokens(ngram, 2)] +=1
        bigram_context[glue_tokens(context, 1)] += 1
print(len(bigrams.keys()), "different bigrams")
print(len(bigram_context.keys()), "different bigram contexts (and unigrams) observed")

15 different bigrams
11 different bigram contexts (and unigrams) observed


In [7]:
# a handy function to calculate the probability of an bigram from the counts
def prob_bigram_MLE(ngram):
    """A simple function to compute the 
    MLE probability estimation based on the counts.
    Follows the equation:
    C(w_i-1, w_i)/C(w_i-1)
    
    Dictionaries bigrams and bigram_context are global variables.

    """
    numerator = bigrams[glue_tokens(ngram, 2)]
    denominator = bigram_context[glue_tokens(ngram[:1], 1)]
    prob = numerator / denominator
    return prob


# check if each bigram continuation distribution sums to one
# look at the distributions of possible continuations after each word
for context, v in bigram_context.items():
    context = unglue_tokens(context, 1)
    print("%% context", context)
    check_ngram_total_sums_to_1 = 0
    # for a given context the continuation probabilities 
    # over the whole vocab should sum to 1
    for u in unigrams.keys():
        ngram = context + [u]
        prob = prob_bigram_MLE(ngram)
        print(ngram, prob)
        check_ngram_total_sums_to_1 += prob
    print("sums to 1?", check_ngram_total_sums_to_1)

%% context ['and']
['and', 'and'] 0.0
['and', 'do'] 0.0
['and', 'like'] 0.0
['and', 'Sam'] 0.0
['and', 'I'] 0.0
['and', 'eggs'] 0.0
['and', 'am'] 0.0
['and', 'green'] 0.0
['and', 'not'] 0.0
['and', '</s>'] 0.0
['and', 'ham'] 1.0
sums to 1? 1.0
%% context ['am']
['am', 'and'] 0.0
['am', 'do'] 0.0
['am', 'like'] 0.0
['am', 'Sam'] 0.5
['am', 'I'] 0.0
['am', 'eggs'] 0.0
['am', 'am'] 0.0
['am', 'green'] 0.0
['am', 'not'] 0.0
['am', '</s>'] 0.5
['am', 'ham'] 0.0
sums to 1? 1.0
%% context ['ham']
['ham', 'and'] 0.0
['ham', 'do'] 0.0
['ham', 'like'] 0.0
['ham', 'Sam'] 0.0
['ham', 'I'] 0.0
['ham', 'eggs'] 0.0
['ham', 'am'] 0.0
['ham', 'green'] 0.0
['ham', 'not'] 0.0
['ham', '</s>'] 1.0
['ham', 'ham'] 0.0
sums to 1? 1.0
%% context ['not']
['not', 'and'] 0.0
['not', 'do'] 0.0
['not', 'like'] 1.0
['not', 'Sam'] 0.0
['not', 'I'] 0.0
['not', 'eggs'] 0.0
['not', 'am'] 0.0
['not', 'green'] 0.0
['not', 'not'] 0.0
['not', '</s>'] 0.0
['not', 'ham'] 0.0
sums to 1? 1.0
%% context ['I']
['I', 'and'] 0.0
['

In [8]:
# Check the estimates for the lecture examples:
# p(I|<s>)
# p(Sam|<s>)
# p(am|I)
# p(</s>|Sam)
# p(Sam|am)
# p(do|I)

print(prob_bigram_MLE(['<s>','I']))
print(prob_bigram_MLE(['<s>', 'Sam']))
print(prob_bigram_MLE(['I', 'am']))
print(prob_bigram_MLE(['Sam', '</s>']))
print(prob_bigram_MLE(['am', 'Sam']))
print(prob_bigram_MLE(['I', 'do']))

0.666666666667
0.333333333333
0.666666666667
0.5
0.5
0.333333333333


To evaluate the model, as in the unigram case above we will measure the model's perplexity of those same training sentences.

Notice that even with this small corpus the bigram perplexity is significantly lower than the unigram perplexity.

In [9]:

s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s
for sent in sentences:
    words = tokenize_sentence(sent, 2)
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for i in range(delta, len(words)):
        context = words[i-delta:i]
        target = words[i]
        ngram = context + [target]
        prob = prob_bigram_MLE(ngram)
        s += -log(prob, 2) # add the neg log prob to s
        sent_s += -log(prob, 2)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N
    sent_perplexity = 2 ** sent_cross_entropy
    print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N
perplexity = 2 ** cross_entropy
print("bigram corpus cross entropy", cross_entropy)
print("bigram corpus perplexity", perplexity)

['<s>', 'I', 'am', 'Sam', '</s>'] cross entropy: 0.792481250361 perplexity: 1.73205080757
['<s>', 'Sam', 'I', 'am', '</s>'] cross entropy: 1.04248125036 perplexity: 2.05976714391
['<s>', 'I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>'] cross entropy: 0.241102777938 perplexity: 1.18189574247
bigram corpus cross entropy 0.559398529666
bigram corpus perplexity 1.47365471155


# Exercises

# Exercise 1. Unigram MLE model from a bigger corpus

Write code to read in the file `switchboard_language_model_train.txt` which has preprocessed text on each line.

You will populate a unigram language model based on that data for an MLE estimation using a `Counter` using this data (see Example 1 above as to how this is done for a smaller dataset).

Before you do this, you have to define a vocabulary of words using the training data, which you will keep the same for all of the following exercises. In these exercises, the vocabulary is defined by using a Minimum Document Frequency of 2 in the training data. That means the vocab should only contain words which occur at least twice in the training data.

Any words not in the vocabulary in the training, heldout and testing data must be replaced with an out-of-vocab symbol `<unk/>` before processing them.
 
Using this model, calculate the perplexity of the ENTIRE test corpus `switchboard_language_model_test.txt`- again, remember to replace words unknown by the model with `<unk/>` before getting these measures. See Example 1 as to how this is done for a unigram model on the smaller example dataset.


# Exercise 2. Bigram model with add-one smoothing

Change your method for reading in and training a language model from Exercise 1 so it works for bigrams. Use the same methods for identifying and replacing out-of-vocabulary words as you did in Exercise 1.

However, in testing, using these raw counts rather than simply implementing MLE you should implement add-one smoothing (see the lecture notes and Jurafsky & Martin Chapter 3/Manning and Schuetze Chapter 6). Remember this involves using the vocabulary size.

Obtain the perplexity score on the test data as above for this smoothed bigram model.

# Exercise 3. Bigram model with general additive (Lidstone) add-k smoothing

Modify your code from Exercise 2 such that it generalizes beyond adding 1 to all counts, but can add differing amounts k of mass instead, to implement general additive add-k smoothing.

On the HELDOUT corpus `switchboard_language_model_heldout.txt` experiment with different values of k (e.g. 0.2, 0.4, 0.6, 0.8, though try others if you can) and report the perplexity scores for these different values in a comment.

Once you find the value which gives you the lowest perplexity on the heldout data, use this model to get the perplexity of the test data once and report the scores in a comment.


# Exercise 4. Ngram models with Kneser-Ney smoothing

Kneser-Ney smoothing is a state-of-the-art technique for smoothing n-gram models.

The algorithm is quite complicated, and is implemented for you below for training/getting the appropriate counts  on the training data (`ngrams_interpolated_kneser_ney()`). The training process is implemented below for a trigram model, but without doing the appropriate out-of-vocab word replacement as you've done above.

The application at test time is done with the method `kneser_ney_ngram_prob()` using the trained Counters, which gives the probability of the model applied to an ngram of a given order, with a given discount.

Try to follow how the training works and how the application of the model to ngrams works, and refer to the below article on QM plus (pages 7-8 particularly):

"A Bit of Progress in Language Modeling" (2001) - Joshua T. Goodman

In this exercise, you will first modify the training part of the code so it does the replacement of out-of-vocab words as you did in the previous exercises. You do not need to modify the methods below.

On the HELDOUT corpus experiment with different orders from trigram upwards (try 3, 4 and 5) and different discount values (e.g. 0.2, 0.4, 0.6, 0.8, though try others if you can) and report the perplexity scores for these different values in a comment. 

Once you find the order and discount values which gives you the lowest perplexity on the heldout data, use this model to get the perplexity of the TEST data once and report the scores in a comment.

In [10]:
# Kneser-Ney smoothing
order = 3
discount = 0.8

unigram_denominator = 0
ngram_numerator_map = Counter() 
ngram_denominator_map = Counter() 
ngram_non_zero_map = Counter()


def ngrams_interpolated_kneser_ney(tokens,
                                   order,
                                   ngram_numerator_map,
                                   ngram_denominator_map,
                                   ngram_non_zero_map,
                                   unigram_denominator):
    """Function used in n-gram language model training
    to count the n-grams in tokens and also record the
    lower order non -ero counts necessary for interpolated Kneser-Ney
    smoothing.
    
    Taken from Goodman 2001 and generalized to arbitrary orders"""
    for i in xrange(order-1,len(tokens)): # tokens should have a prefix of order - 1
        #print i
        for d in xrange(order,0,-1): #go through all the different 'n's
            if d == 1:
                unigram_denominator += 1
                ngram_numerator_map[glue_tokens(tokens[i],d)] += 1
            else:
                den_key = glue_tokens(tokens[i-(d-1) : i], d)
                num_key = glue_tokens(tokens[i-(d-1) : i+1], d)
    
                ngram_denominator_map[den_key] += 1
                # we store this value to check if it's 0
                tmp = ngram_numerator_map[num_key]
                ngram_numerator_map[num_key] += 1 # we increment it
                if tmp == 0: # if this is the first time we see this ngram
                    #number of types it's been used as a context for
                    ngram_non_zero_map[den_key] += 1
                else:
                    break 
                    # if the ngram has already been seen
                    # we don't go down to lower order models
    return ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator


In [11]:
# train the model
corpus = open("switchboard_lm_train.txt")
for line in corpus:
    tokens = tokenize_sentence(line, order)
    ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator =\
            ngrams_interpolated_kneser_ney(tokens,
                                           order,
                                           ngram_numerator_map,
                                           ngram_denominator_map,
                                           ngram_non_zero_map,
                                           unigram_denominator)
corpus.close() 

In [12]:
def kneser_ney_ngram_prob(ngram, discount, order):
    """KN smoothed ngram probability from Goodman 2001.
    This is run at test time to calculate the probability
    of a given n-gram or a given order with a given discount.
    
    ngram :: list of strings, the ngram
    discount :: float, the discount used (lambda)
    order :: int, order of the model
    """
    # First, calculate the unigram prob of the last token 
    # If we've never seen it at all, it will 
    # have no probability as a numerator
    uni_num = ngram_numerator_map.get(glue_tokens(ngram[-1], 1))
    if not uni_num: # if no value found in dict, make it 0
        uni_num = 0
    probability = previous_prob = float(uni_num) / float(unigram_denominator)
    
    # Given <unk/> should have been used in place of unknown words before passing
    # to this method,
    # probability should be non-zero
    if probability == 0.0:
        print("0 prob for unigram!")
        print(glue_tokens(ngram[-1], 1))
        print(ngram)
        print(ngram_numerator_map.get(glue_tokens(ngram[-1], 1)))
        print(unigram_denominator)
        raise Exception

    # Compute the higher order probs (from 2/bi-gram upwards) and interpolate them
    for d in xrange(2,order+1):
        # Get the number of times this denominator has been seen as one
        # For bigrams this is the number of different continuation types counted
        ngram_den = ngram_denominator_map.get(glue_tokens(ngram[-(d):-1], d))
        if not ngram_den: # if no value found in dict, make it 0
            ngram_den = 0
        if ngram_den != 0: 
            ngram_num = ngram_numerator_map.get(glue_tokens(ngram[-(d):], d))
            if not ngram_num: # if no value found in dict, make it 0
                ngram_num = 0
            if ngram_num != 0:
                current_prob = (ngram_num - discount) / float(ngram_den)
            else:
                current_prob = 0.0
            nonzero = ngram_non_zero_map.get(glue_tokens(ngram[-(d):-1], d))
            if not nonzero: # if no value found in dict, make it 0
                nonzero = 0
            # interpolate with previous probability of lower orders calculated
            # so far
            current_prob += nonzero * discount / ngram_den * previous_prob
            previous_prob = current_prob
            probability = current_prob
        else:
            #if this context (e.g. bigram contect for trigrams) has never been seen, 
            #then we can only get the last order with a probability (e.g. unigram)
            #and halt
            probability = previous_prob
            break
    return probability